In [ ]:
import numpy as np
from numpy import array
import pandas as pd
import random
from random import randint
from pickle import dump, load
from sklearn.model_selection import train_test_split
import sys
import string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, LSTM, Embedding
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import textstat
import nltk
from itertools import compress, cycle, islice
from sklearn.metrics.pairwise import cosine_similarity

# i'm not even using these anymore
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

random.seed(952)


def clean_text(input):
    # tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)

    # remove punctuation
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]

    # remove non alphabetic
    tokens = [word for word in tokens if word.isalpha()]

    # make lower case
    tokens = [word.lower() for word in tokens]

    # remove tokens of length 1
    tokens_len = [len(i) > 1 for i in tokens]
    tokens_filter = list(compress(tokens, tokens_len))
    tokens = tokens_filter

    return tokens


# save tokens to file, one sequence per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()


def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text

    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]

        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')

        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)

        # map predicted word index to word
        out_word = ''

        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break

        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)


def detect_labels(path):
    """Detects labels in LOCAL file."""
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.label_detection(image=image)
    labels = response.label_annotations

    # list of labels (ignoring uncertainty)
    labels = [x.description for x in labels]
    return labels

In [129]:
# load ikea catalogue
ikea_cat = pd.read_csv('../results/ikea_2.csv')

ikdes = ikea_cat.description

tester = ['bookcase', 'shelf', 'hutch']

# find which descriptions have any of the right words
ikea_words = []
for ii in range(0, len(ikdes)):
    ct = clean_text(str((ikdes[ii])))
    ikea_words.append(any(item in tester for item in ct))

     

In [130]:
def replace_nouns(text, replace):
    tokenized = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(tokenized)
    
    tt = []
    for ii in range(0, len(tagged)):
        tt.append(tagged[ii][1][0] == 'N')
        
    replacements = list(islice(cycle(tester), sum(tt)))
    
    jj = 0
    for ii in range(0, len(tagged)):
        if tt[ii]:
            tokenized[ii] = replacements[jj]
            jj = jj + 1
            
    return ' '.join(tokenized)

Clearly visible chunks of chickpeas, green peas, carrots, bell peppers, corn and kale in a beautiful array of colors. An eating option with fewer calories and less fat. By eating more vegetables you reduce your carbon footprint. Includes no animal ingredients, therefore it's a good choice for vegetarians and vegans. Veggie balls should taste like vegetables. That’s why ours consist of nothing but vegetables. They’re a celebration of vegetables. The recipe does not contain ingredients with allergens such as gluten, soy, nut, egg or lactose so as many people as possible can enjoy them. A good source of fiber, protein, vitamins and nutrients, but low in cholesterol. Easily prepared from frozen in minutes. In the oven, in the microwave or on the stove. Easily portioned. Prepare the amount needed, leave the rest in the freezer. Can be combined in many different ways. For example, try these instead of meatballs in your favorite meatball dish. Just as tasty warm as they are cold, e.g. cut in 